In [2]:
# import necessary libraries

import random
import os, pickle
import numpy as np
from collections import OrderedDict

In [8]:
# reuse processed or synthesized data from dcan

data_path = '../dcan/data/syn/12m'
out_path = 'data/syn/12m'

names = os.listdir(data_path)
for name in names:
    # load image and segmentation
    data = np.load(os.path.join(data_path, name))
    seg = data[-1]
    # compensate label imbalance
    approx_nsamp = 10000
    samp_locs = OrderedDict()
    for cls in [1, 2, 3]:
        locs = np.argwhere(seg == cls)
        nsamp = min(approx_nsamp, len(locs))
        nsamp = max(nsamp, int(np.ceil(0.1 * len(locs))))
        samp = locs[random.sample(range(len(locs)), nsamp)]
        if len(samp) != 0:
            samp_locs[cls] = samp
    with open(os.path.join(out_path, name[:-4]+'.pkl'), 'wb') as f:
        pickle.dump(samp_locs, f)
    np.save(os.path.join(out_path, name), data)

In [3]:
# split dataset into train, valid, and test set

data_path = 'data/processed/6m'

names = os.listdir(data_path)
random.shuffle(names)

splits = OrderedDict()
splits['train'] = names[:int(len(names)*0.7)]
splits['valid'] = names[int(len(names)*0.7):int(len(names)*0.8)]
splits['test'] = names[int(len(names)*0.8):]

with open(os.path.join(data_path, 'splits.pkl'), 'wb') as f:
    pickle.dump(splits, f)